In [10]:
import sympy as sp
from ham_to_matrix import *
from constants import *

## Diagonal Part

In [19]:
totHam = 0.5*(pn**2 + qn**2)

In [20]:
totHam

0.5*(p_{n}**2 + q_{n}**2)

In [21]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}

# warning be careful of array indexing...   
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1]
                         }
elif bcType == 'dirichlet':
    boundaryConditions = {qs[0]: 0, qs[N+1]: 0}

In [22]:
ham=0
for i in range(1,N+1):
    ham+=totHam.subs({qn: qs[i], pn: ps[i]}).subs(boundaryConditions)

ham

0.5*(p_{0}**2 + q_{0}**2) + 0.5*(p_{1}**2 + q_{1}**2) + 0.5*(p_{2}**2 + q_{2}**2)

In [23]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

a^{\dagger}_{0}*a_{0}/2 + a^{\dagger}_{1}*a_{1}/2 + a^{\dagger}_{2}*a_{2}/2 + a_{0}*a^{\dagger}_{0}/2 + a_{1}*a^{\dagger}_{1}/2 + a_{2}*a^{\dagger}_{2}/2

## H = n_0 + 1/2 + n_1 + 1/2

In [24]:
cutoff=4
term=hoHam.args[0]+hoHam.args[3]
#term+=hoHam.args[1]+hoHam.args[4]
print(term)
hamMat=convert_boson_to_matrix(term,cutoff,3,aops,adags)
print(hamMat)
ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
ens=ens.round(8)

np.isreal(ens).all()

ens

a^{\dagger}_{0}*a_{0}/2 + a_{0}*a^{\dagger}_{0}/2
[[0.500000000000000 0 0 ... 0 0 0]
 [0 0.500000000000000 0 ... 0 0 0]
 [0 0 0.500000000000000 ... 0 0 0]
 ...
 [0 0 0 ... 1.50000000000000 0 0]
 [0 0 0 ... 0 1.50000000000000 0]
 [0 0 0 ... 0 0 1.50000000000000]]


array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j,
       0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j,
       0.5+0.j, 0.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j,
       1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j,
       1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j,
       1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j,
       1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 2.5+0.j,
       2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j,
       2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j,
       2.5+0.j], dtype=complex64)

In [25]:
cutoff=4
hamMat=convert_boson_to_matrix(hoHam,cutoff,N,aops,adags)
ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
ens=ens.round(8)

np.isreal(ens).all()

ens

array([1.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j,
       3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j,
       3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j,
       3.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j,
       4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j,
       4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j, 4.5+0.j,
       5.5+0.j, 5.5+0.j, 5.5+0.j, 5.5+0.j, 5.5+0.j, 5.5+0.j, 5.5+0.j,
       5.5+0.j, 5.5+0.j, 5.5+0.j, 5.5+0.j, 5.5+0.j, 5.5+0.j, 5.5+0.j,
       5.5+0.j, 6.5+0.j, 6.5+0.j, 6.5+0.j, 6.5+0.j, 6.5+0.j, 6.5+0.j,
       7.5+0.j], dtype=complex64)

In [29]:
cutoff=4
hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,[0,0,0,0,0],[0,0,0,0,0])
ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
ens=ens.round(8)

np.isreal(ens).all()

ens

a^{\dagger}_{0}*a_{0}/2 to matrix timer took: 48.9547 seconds
a^{\dagger}_{1}*a_{1}/2 to matrix timer took: 35.2551 seconds
a^{\dagger}_{2}*a_{2}/2 to matrix timer took: 34.8753 seconds
a_{0}*a^{\dagger}_{0}/2 to matrix timer took: 34.5578 seconds
a_{1}*a^{\dagger}_{1}/2 to matrix timer took: 40.2218 seconds
a_{2}*a^{\dagger}_{2}/2 to matrix timer took: 34.1281 seconds


array([1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j, 1.5+0.j,
       1.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j,
       2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j,
       2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j,
       2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j,
       2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j,
       2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j,
       2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j, 2.5+0.j,
       3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j,
       3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j,
       3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j,
       3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j,
       3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j,
       3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j, 3.5+0.j,
       3.5+0.j, 3.5+

## Full boson part

In [38]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(2*aLat) )


totHam=bosonNI+bosonI

In [39]:
totHam

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [40]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}

# warning be careful of array indexing...   
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1]
                         }
elif bcType == 'dirichlet':
    boundaryConditions = {qs[0]: 0, qs[N+1]: 0}

In [41]:
ham=0
for i in range(1,N+1):
    ham+=totHam.subs({qn: qs[i], pn: ps[i], qnP1: qs[i+1], qnM1: qs[i-1]}).subs(boundaryConditions)

ham

a*V(q_{0})**2/2 + a*V(q_{1})**2/2 + a*V(q_{2})**2/2 + V(q_{0})*q_{1}/2 - V(q_{0})*q_{2}/2 - V(q_{1})*q_{0}/2 + V(q_{1})*q_{2}/2 + V(q_{2})*q_{0}/2 - V(q_{2})*q_{1}/2 + p_{0}**2/(2*a) + p_{1}**2/(2*a) + p_{2}**2/(2*a) - q_{0}*q_{1}/(8*a) - q_{0}*q_{2}/(8*a) + q_{0}**2/(4*a) - q_{1}*q_{0}/(8*a) - q_{1}*q_{2}/(8*a) + q_{1}**2/(4*a) - q_{2}*q_{0}/(8*a) - q_{2}*q_{1}/(8*a) + q_{2}**2/(4*a)

In [45]:
def potential(n):
    return 0

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

aVal=1
    
ham=ham.subs(potentialSubs).subs(aLat,aVal)
ham

p_{0}**2/2 + p_{1}**2/2 + p_{2}**2/2 - q_{0}*q_{1}/8 - q_{0}*q_{2}/8 + q_{0}**2/4 - q_{1}*q_{0}/8 - q_{1}*q_{2}/8 + q_{1}**2/4 - q_{2}*q_{0}/8 - q_{2}*q_{1}/8 + q_{2}**2/4

In [46]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

-a^{\dagger}_{0}*a^{\dagger}_{1}/16 - a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 3*a^{\dagger}_{0}*a_{0}/8 - a^{\dagger}_{0}*a_{1}/16 - a^{\dagger}_{0}*a_{2}/16 - a^{\dagger}_{0}**2/8 - a^{\dagger}_{1}*a^{\dagger}_{0}/16 - a^{\dagger}_{1}*a^{\dagger}_{2}/16 - a^{\dagger}_{1}*a_{0}/16 + 3*a^{\dagger}_{1}*a_{1}/8 - a^{\dagger}_{1}*a_{2}/16 - a^{\dagger}_{1}**2/8 - a^{\dagger}_{2}*a^{\dagger}_{0}/16 - a^{\dagger}_{2}*a^{\dagger}_{1}/16 - a^{\dagger}_{2}*a_{0}/16 - a^{\dagger}_{2}*a_{1}/16 + 3*a^{\dagger}_{2}*a_{2}/8 - a^{\dagger}_{2}**2/8 + 3*a_{0}*a^{\dagger}_{0}/8 - a_{0}*a^{\dagger}_{1}/16 - a_{0}*a^{\dagger}_{2}/16 - a_{0}*a_{1}/16 - a_{0}*a_{2}/16 - a_{0}**2/8 - a_{1}*a^{\dagger}_{0}/16 + 3*a_{1}*a^{\dagger}_{1}/8 - a_{1}*a^{\dagger}_{2}/16 - a_{1}*a_{0}/16 - a_{1}*a_{2}/16 - a_{1}**2/8 - a_{2}*a^{\dagger}_{0}/16 - a_{2}*a^{\dagger}_{1}/16 + 3*a_{2}*a^{\dagger}_{2}/8 - a_{2}*a_{0}/16 - a_{2}*a_{1}/16 - a_{2}**2/8

In [47]:
cutoff=4

hamMat=convert_boson_to_matrix(hoHam,cutoff,3,aops,adags)
print(hamMat)
ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
ens=ens.round(8)

np.isreal(ens).all()

ens

[[1.12500000000000 0 -0.176776692271233 ... 0 0 0]
 [0 1.87499995529652 0 ... 0 0 0]
 [-0.176776692271233 0 2.62499995529652 ... 0 0 0]
 ...
 [0 0 0 ... 3.37499995529652 0 -0.306186228990555]
 [0 0 0 ... 0 4.12499995529652 0]
 [0 0 0 ... -0.306186228990555 0 3.37500000000000]]


array([0.8653307 +0.j, 0.91822773+0.j, 1.20834   +0.j, 1.4751848 +0.j,
       1.4751848 +0.j, 1.5908326 +0.j, 1.6946908 +0.j, 1.6946908 +0.j,
       2.044932  +0.j, 2.1453073 +0.j, 2.1814473 +0.j, 2.1814473 +0.j,
       2.3267992 +0.j, 2.3267992 +0.j, 2.3393168 +0.j, 2.3393168 +0.j,
       2.5307605 +0.j, 2.5807838 +0.j, 2.5807838 +0.j, 2.6289666 +0.j,
       2.75      +0.j, 2.8610654 +0.j, 2.8943396 +0.j, 2.8943396 +0.j,
       3.0365508 +0.j, 3.0365508 +0.j, 3.0659428 +0.j, 3.1264853 +0.j,
       3.2905416 +0.j, 3.2905416 +0.j, 3.3886526 +0.j, 3.3886526 +0.j,
       3.5273242 +0.j, 3.6005867 +0.j, 3.6207943 +0.j, 3.6207943 +0.j,
       3.6527712 +0.j, 3.6527712 +0.j, 3.75      +0.j, 3.75      +0.j,
       3.8454235 +0.j, 3.8454235 +0.j, 3.9405062 +0.j, 3.9738288 +0.j,
       3.9738288 +0.j, 4.0834284 +0.j, 4.0834284 +0.j, 4.0928097 +0.j,
       4.304533  +0.j, 4.40088   +0.j, 4.444488  +0.j, 4.444488  +0.j,
       4.702984  +0.j, 4.702984  +0.j, 4.75      +0.j, 4.783016  +0.j,
      